##### 6427 images in training set, 1070 in test set.
##### label: 
##### 0 -> Normal, which is 0 in the image
##### 1 -> Anomaly, image is not 0
##### Training label given, need to predict output of test set

# CNN

In [29]:
# load packages
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
import os
from tqdm import tqdm as tqdm

from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [30]:
# To load images to features and labels
def load_images_to_data(image_directory):
    list_of_files = os.listdir(image_directory)
    features_data = np.empty((0,28,28,1),dtype="float32")
    for file in tqdm(list_of_files):
        image_file_name = os.path.join(image_directory, file)
        if ".jpg" in image_file_name:
            img = Image.open(image_file_name)
            img = np.resize(img, (28,28,1))
            im2arr = np.array(img)
            im2arr = im2arr.reshape(1,28,28,1)
            features_data = np.append(features_data, im2arr, axis=0)
    return features_data

In [31]:
train_directory = './bt4012-in-class-competition/train_images/'
test_directory = './bt4012-in-class-competition/test_images/'
train_label_directory = './bt4012-in-class-competition/train.csv'
#pre_label_directory = './bt4012-in-class-competition/test.csv'

In [32]:
X = load_images_to_data(train_directory)
y = pd.read_csv(train_label_directory)['label'].values.astype(np.int)
X_pred = load_images_to_data(test_directory)
#y_pred = pd.read_csv(pre_label_directory)['label'].values.astype(np.int)

100%|██████████| 1070/1070 [00:02<00:00, 472.31it/s]


In [33]:
# split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [34]:
# add in more data from library
from keras.datasets import mnist
(trainX, trainy), (testX, testy) = mnist.load_data()

# reshape new data
trainX = trainX.reshape(trainX.shape[0], 28, 28, 1).astype('float32')
testX = testX.reshape(testX.shape[0], 28, 28, 1).astype('float32')
trainy = np.array(list(map(lambda x: 0 if x==0 else 1, trainy)))
testy = np.array(list(map(lambda x: 0 if x==0 else 1, testy)))

In [35]:
# append
X_train = np.append(X_train, trainX, axis = 0)
X_test = np.append(X_test, testX, axis = 0)
y_train = np.append(y_train, trainy, axis = 0)
y_test = np.append(y_test, testy, axis = 0)

In [36]:
# undersampling
y_train_df = pd.DataFrame(y_train, columns=['y'])
positive_class = y_train_df[y_train_df.y==1].index.tolist()
negative_class = y_train_df[y_train_df.y==0].index.tolist()
positive_class = positive_class[0:10000]
positive_class.extend(negative_class)
X_train = X_train[positive_class]
y_train = y_train[positive_class]

In [37]:
# scale down the data 
X_train /= 255
X_test /= 255
X_pred /= 255

In [38]:
input_shape = (28, 28, 1)

In [39]:
number_of_classes = 2
y_train = np_utils.to_categorical(y_train, number_of_classes)
y_test = np_utils.to_categorical(y_test, number_of_classes)
y_pred = np_utils.to_categorical(y_pred, number_of_classes)

In [42]:
# create model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(X_train.shape[1], X_train.shape[2], 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) 

In [44]:
# compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=[tf.keras.metrics.AUC()])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=200)

Epoch 1/50
104/104 [==============================] - 15s 142ms/step - loss: 0.0939 - auc_11: 0.9930 - val_loss: 0.4928 - val_auc_11: 0.8614
Epoch 2/50
104/104 [==============================] - 13s 125ms/step - loss: 0.0883 - auc_11: 0.9935 - val_loss: 0.4011 - val_auc_11: 0.8900
Epoch 3/50
104/104 [==============================] - 12s 116ms/step - loss: 0.0840 - auc_11: 0.9942 - val_loss: 0.3570 - val_auc_11: 0.9049
Epoch 4/50
104/104 [==============================] - 12s 117ms/step - loss: 0.0821 - auc_11: 0.9944 - val_loss: 0.3860 - val_auc_11: 0.8870
Epoch 5/50
104/104 [==============================] - 11s 106ms/step - loss: 0.0818 - auc_11: 0.9942 - val_loss: 0.4086 - val_auc_11: 0.8823
Epoch 6/50
104/104 [==============================] - 11s 105ms/step - loss: 0.0806 - auc_11: 0.9946 - val_loss: 0.5135 - val_auc_11: 0.8525
Epoch 7/50
104/104 [==============================] - 11s 106ms/step - loss: 0.0783 - auc_11: 0.9947 - val_loss: 0.5528 - val_auc_11: 0.8509
Epoch 8/50
10

In [45]:
# make prediction
model.evaluate(X_test, y_test)

353/353 [==============================] - 2s 7ms/step - loss: 0.0467 - auc_11: 0.9977


[0.04665688797831535, 0.9977399110794067]

In [23]:
y_prediction = model.predict_classes(X_pred)
prediction = pd.DataFrame(y_prediction, columns=['target']).to_csv('/Users/yimingzhao/Desktop/prediction.csv')

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


# SGD (Ignore first)

In [54]:
# Creating a Stochastic Gradient Descent(SGD) classifier
# https://www.kaggle.com/sikora507/mnist-binary-classifier-precision-and-recall
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.model_selection import cross_val_score
import cv2 

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [108]:
X_train.shape

(5141, 28, 28, 1)

In [111]:
X_train = tf.reshape(X_train, [4030544, 1])

In [112]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

skfolds = StratifiedKFold(n_splits=3, random_state=42)

i=0;
for train_index, test_index in skfolds.split(X_train, y_train):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print("Correct ratio for fold {1}: {0}".format(n_correct / len(y_pred), i))
    i += 1


ValueError: Found input variables with inconsistent numbers of samples: [4030544, 5141]

In [ ]:
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")